In [1]:
!pip install torch datasets

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

In [4]:
# Creating a student model over here using bert model
STUDENT_MODEL = "google-bert/bert-base-uncased"

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(STUDENT_MODEL)
model

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [5]:
dataset = load_dataset("sst2", split="train[:10]")


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [6]:
dataset

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 10
})

In [7]:
!pip install langchain_ollama # installing the ollam model

In [19]:
from langchain_ollama import ChatOllama

# Teacher Model
llm_engine = ChatOllama(
    model="llama2",
    base_url = "127.0.0.1:11434",
    temperature = 0.3
  )

In [20]:
test_prompt = "I am so happy today"
response = llm_engine.invoke(test_prompt)
print(response.content)

ConnectError: [Errno 111] Connection refused

In [5]:
# Generates a step-by-step rationale before answering using chain-of-thought (COT) prompting

def generate_rationale(input_text):
  """ Uses ollama's DeepSeek R1 model to generate a step-by-step rationale for the given input."""

  prompt = f"Explain step-by-step reasoning before answering: {input_text}" # cot prompting req for step by step distil

  response = llm_engine.invoke(prompt)

  return response.content if hasattr(response, 'content') else response

  print(generate_rationale("Explain AI in one sentence"))











In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL)

NameError: name 'STUDENT_MODEL' is not defined

In [4]:
# prepare dataset with rationales

def process_data(example):
  input_text = example['sentence'] # change this depending on ur dataset format
  rationale = generate_rationale(input_text)
  label = example['label']

  # Tokenize input and rationale
  input_encoding = tokenizer(input_text, truncation=True, padding='max_length', max_length=256)
  rationale_encoding = tokenizer(rationale, truncation=True, padding='max_length', max_length=256)
  return {
      "input_ids": input_encoding["input_ids"],
      "attention_mask": input_encoding["attention_mask"],
      "rationale_ids": rationale_encoding["input_ids"],
      "rationale_attention_mask": rationale_encoding["attention_mask"],
      "label": label
      }


# Apply function to dataset
proecessed_dataset = dataset.map(process_data)






NameError: name 'dataset' is not defined

check out video 19th and here we need to run our ollma locally add url path of it and well run all the code